In [1]:
from ipyleaflet import Map, LocalTileLayer, TileLayer, LayersControl, FeatureGroup, LayerGroup, basemaps, basemap_to_tiles, WidgetControl

In [2]:
import ipywidgets as widgets

In [3]:
from ipywidgets import Layout

In [4]:
from IPython.display import display

In [5]:
chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

In [6]:
m = Map(center=(13.478, -15.125), zoom=6)

In [7]:
water_risk_layers = []
population_risk_layers = []
absolute_risk_layers = []
relative_risk_layers = []
checkbox_layers_map = {}

In [8]:
# Add NDVI layers
for season in ['dry', 'wet']:
    ndvi_tile_url = TileLayer(
        url=f'http://localhost:8080/output/ndvi_suit_{season}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"ndvi {season}",
        attribution="mine",
        min_zoom=1,
        max_zoom=16,
        min_native_zoom=1,
        max_native_zoom=14,)
    water_risk_layers.append(f'ndvi_suit_{season}')
    checkbox_layers_map[f'ndvi_suit_{season}'] = ndvi_tile_url

In [9]:
# Add water temperature layers
for risk_type in ['parasite', 'snail']:
    for season in ['dry', 'wet']:
        temp_tile_url = TileLayer(
            url=f'http://localhost:8080/output/{risk_type}_water_temp_suit_{season}_tiles/{{z}}/{{x}}/{{y}}.png',
            name=f"{risk_type} water temp {season}",
            attribution="mine",
            min_zoom=1,
            max_zoom=16,
            min_native_zoom=1,
            max_native_zoom=14,)
        water_risk_layers.append(f'{risk_type}_water_temp_suit_{season}')
        checkbox_layers_map[f'{risk_type}_water_temp_suit_{season}'] = temp_tile_url

In [10]:
# Add population risk layers
population_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/rural_urbanization_suit_tiles/{z}/{x}/{y}.png',
    name="Population risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
population_risk_layers.append('rural_urbanization_suit')
checkbox_layers_map['rural_urbanization_suit'] = population_risk_tile_url

In [11]:
# Add weighted mean risk layer
combined_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/habitat_suit_weighted_mean_tiles/{z}/{x}/{y}.png',
    name="Combined risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
water_risk_layers.append('habitat_suit_weighted_mean')
checkbox_layers_map['habitat_suit_weighted_mean'] = combined_risk_tile_url

In [12]:
# Add distance weighted risk (convolution layer)
convolved_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/convolved_hab_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
absolute_risk_layers.append('convolved_hab_risk')
checkbox_layers_map['convolved_hab_risk'] = convolved_risk_tile_url

In [13]:
# Add normalized distance weighted risk
normalized_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/normalized_convolved_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
relative_risk_layers.append('normalized_convolved_risk')
checkbox_layers_map['normalized_convolved_risk'] = normalized_risk_tile_url

In [14]:
# Add relative and absolute risk to population outputs
for risk_type, layer_group in zip(['abs', 'rel'], [absolute_risk_layers, relative_risk_layers]):
    risk_to_pop_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=16,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_{risk_type}'] = risk_to_pop_tile_url
    
    risk_to_pop_count_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_count_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop count {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=16,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_count_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_count_{risk_type}'] = risk_to_pop_count_tile_url

In [15]:
out = widgets.Output()

@out.capture()
def layer_visible_switch(event):
    widget = event['owner']
    desc_id = widget.description
    print(desc_id)
    layer_tile_url = checkbox_layers_map[desc_id]
    layer_tile_url.visible = event['new']

layer_groups = [water_risk_layers, population_risk_layers, absolute_risk_layers, relative_risk_layers]
layer_group_sections = ["Water risk layers", "Population risk layers", "Absolute risk layers", "Relative risk layers"]
widget_list = []
for group_title, group_list in zip(layer_group_sections, layer_groups):
    group_header_widget = widgets.HTML(description="", value=f'<b>{group_title}<b>')
    widget_list.append(group_header_widget)
    for layer_key in group_list:
        tmp_widget = widgets.Checkbox(
                value=False,
                description=layer_key,
                disabled=False,
                indent=False,
                layout=Layout(padding='0px 0px 0px 10px', margin='0px 0px 0px 0px'),
            )
        tmp_widget.observe(layer_visible_switch, names='value')
        widget_list.append(tmp_widget)
        checkbox_layers_map[layer_key].visible=False
        m.add(checkbox_layers_map[layer_key])
        
vbox_widget = widgets.VBox(widget_list, layout=Layout(height='300px', overflow_y='auto', border='solid', padding='0px 0px 0px 5px'))

display(vbox_widget)
display(out)


Output()

In [16]:
widget_control = WidgetControl(widget=vbox_widget, position="bottomleft")

In [17]:
m.add(widget_control)

Map(center=[13.478, -15.125], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [ ]:
# use the LayersControl to switch basemaps
#m.add_control(LayersControl())

In [ ]:
tmp_widget = widgets.Checkbox(
                value=False,
                description='layer_key',
                disabled=False,
                indent=True
            )
tmp_widget.style.keys

In [ ]:
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from os import walk
plot_paths = 'http://localhost:8888/intermediate/plot_previews'
#onlyfiles = [f for f in listdir(plot_paths) if isfile(join(plot_paths, f))]
onlyfiles = []
for dir_path, dir_names, file_names in walk(plot_paths):
    onlyfiles = [join(dir_path, f) for f in file_names]
    


In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(14, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

fig.suptitle("Suitability function plots")
for ax, image_file in zip(axs.flat, onlyfiles):
    img = plt.imread(image_file)
    #ax.imshow(grid, interpolation=interp_method, cmap='viridis')
    ax.imshow(img)
    
plt.tight_layout()
plt.show()
